In [1]:
#Usual Imports
import os
import sys
import time
import datetime
import re
import numpy as np
import pandas as pd
# Imports for web scraping Spotify data
from bs4 import BeautifulSoup
import urllib
import requests
from __future__ import print_function    # (at top of module)
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from time import sleep
from collections import Counter

In [2]:
# Initialize Spotipy API credentials
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False

In [3]:
# Read in billboard top 100 data from 1955-2015
billboard_data=pd.read_csv("all_billboard_data.txt", sep="|", encoding='latin1')
#Arrange Billboard data by tracks for Spotify web scraping 
billboard_tracks = billboard_data.groupby(["title","artist"]).size().reset_index()

In [4]:
# We have 34605 unique songs in our dataset
len(billboard_tracks)

34605

In [8]:
# Initialize an empty data frame for the Spotify data
track_meta_columns=["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","type","id","uri","track_href","analysis_url","duration_ms","time_signature"]
artist_data = pd.DataFrame(columns=["artist","track"])
track_data = pd.DataFrame(columns=track_meta_columns)
spotify_data=pd.concat([artist_data, track_data], axis=1)
artist_index=0
track_index=0

In [6]:
# Obtaining Data Step
spotify_data=pd.concat([artist_data, track_data], axis=1)
lower_limit =0
upper_limit =150
spot_table = 'spotify' +str(lower_limit)+"-"+ str(upper_limit)+'.csv'
start = time.time()
# Cycle through the rows in the billboard_tracks dataframe.
# Search for the artist and title for each on through Spotify's API
# Add the metadata for the track to the dataset
for row in range(lower_limit,upper_limit):
    artist_data = pd.DataFrame(columns=["artist","track"])
    track_data = pd.DataFrame(columns=track_meta_columns)
    artist_name = billboard_tracks.iloc[row,1]
    track_name = billboard_tracks.iloc[row,0]
    lookup = artist_name + " " + track_name
    try:
        results = sp.search(q=lookup, limit=3)
    except urllib.request.HTTPError as eone:
        pass
    tids = []
    artist_index=0
    for i, t in enumerate(results['tracks']['items']):
        to_add =pd.DataFrame( {"artist":t['artists'][0]['name'], "track":t['name']},index=[artist_index])
        artist_data=pd.concat([artist_data,to_add])
        tids.append(t['uri'])
        artist_index+=1
    try:    
        features = sp.audio_features(tids)
    except urllib.request.HTTPError as etwo:
        pass    
    track_index=0
    for feature in features:
        tracks = pd.DataFrame(feature, index=[track_index])
        track_data=pd.concat([track_data,tracks])
        track_index+=1
    all_data = pd.concat([artist_data, track_data], axis=1)
    spotify_data=pd.concat([spotify_data,all_data])
# Spotify has rate limiting for their API so we must delay time between requests
    if row%50==0 and row!=lower_limit and row!=upper_limit:
        sleep(25)
delta = time.time() - start
print ("features retrieved in %.2f seconds" % (delta,))
# Output the data to a csv file
# Note that due to the length of time required we will create several csv files
spotify_data.to_csv(spot_table)

retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
retrying ...0secs
features retrieved in 113.41 seconds


In [ ]:
# Cleaning and Wrangling step
# Begin to join all the csv files together, some may require adding in an additional empty row.
spot_one = pd.read_csv('spotify0-50.csv',  encoding='latin1')
spot_two = pd.read_csv('spotify50-100.csv',  encoding='latin1')
spotify_all = pd.concat([spot_one,spot_two])
spot_new = pd.read_csv('spotify100-200.csv',  encoding='latin1')
spotify_all = pd.concat([spotify_all,spot_new])
spotify_empty = spot_small[87:88]
spotify_all = pd.concat([spotify_all,spotify_empty])
# spotify_all.to_csv("spotify_big")

In [5]:
# Read in Large Spotify dataset
big_spotify = pd.read_csv('spotify_big.csv',  encoding='latin1')
# We will match the billboard_track data and the spotify data one-to-one
# We only require one result per track
spotify_merge = big_spotify[big_spotify["Unnamed: 0.1"]==0]
# Subset interesting columns
spotify_merge =spotify_merge[["acousticness","artist","danceability","duration_ms","energy","key","loudness","mode","track","instrumentalness","liveness","valence","tempo","time_signature"]]
# Rename artist column
spotify_merge=spotify_merge.rename(columns = {'artist':'spotify_artist'})
spotify_merge=spotify_merge.reset_index()
len(spotify_merge)

34607

In [6]:
# Drop two extra rows
spotify_merge=spotify_merge.drop(spotify_merge.index[[101,201]]).reset_index()
# Merge data with the billboard_data
merged_data=pd.concat([billboard_tracks, spotify_merge], axis=1)
# Join data to billboard_list
all_data=pd.merge(billboard_data,merged_data,how='left', on=['artist', 'title'])

In [12]:
len(all_data)

343545

In [ ]:
spotify_nonas = spotify_all.dropna(subset=["artist"])
# Replace all Karaoke data with NA's

# Big artists not on Spotify
No_Spotify_list = ["TAYLOR SWIFT", "PRINCE", "GARTH BROOKS", "TOOL", "BOB SEGAR", "KING CRIMSON", "TOM YORKE", "TRAVELING WILBURYS"]

cleaner_data=spotify_nonas[spotify_nonas['artist'].str.contains("Karaoke")!=True]

In [ ]:
#### Remember to remove "We do not have the lyrics yet" from lyrics column and starts with CHORUS

In [ ]:
# Analysis

In [3]:
new_big_data= pd.read_csv("all_charts.csv")

C:\Users\MSI_USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,19,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
search_data=new_big_data.groupby(["title","artist"]).size().reset_index()

In [235]:
len(search_data)

27588

In [33]:
lyrics_data = pd.DataFrame()
lower_limit =27587
upper_limit =27588
lyrics_table = 'lyrics-' +str(lower_limit)+"-"+ str(upper_limit)+'.csv'
start = time.time()
for row in range(lower_limit,upper_limit):
    artist_raw = search_data.iloc[row,1]
    artist_noand=artist_raw.replace(" &","").replace(". ","-").replace("Ã©","e").replace("$", "s")
    artist_array = re.split(r'.Featuring+', artist_noand)
    track_raw=search_data.iloc[row,0]
    # Replace extended latin characters
    track_name=track_raw.replace("Ã\x83Â¶","oe").replace("Ã©","e").replace("ÃÂ©","e").replace("ÃÂ³",'o')
    track_name=track_name.replace("ÃÂ¡","a").replace("ÃÂ±","n").replace("Ã\x83â\x80\x9c","o").replace("ÃÂ¿DÃÂ²","do").replace("ÃÂ","a")
    track_name=re.sub(r'\&\s', '', track_name).replace("$", "s").replace("#","") # Replace certain characters
    track_name=re.sub(r'S\*\*t', 'Shit',track_name) # Replace profanity
    track_name=re.sub(r'\(|\)|\'|\|\.|\"|\,', '',track_name).lower() # Remove special characters
    # We try five different lyrics sites
    artist_first_letter=artist_array[0][0]
    artist_mode = re.sub(r'\s|\.|-', '_',artist_array[0]).replace(",","").lower()
    track_mode = re.sub(r'\s', '_',track_name)
    artist_song = re.sub(r'\s|\.', '-',artist_array[0]).replace(",","").lower()
    track_songmetro = re.sub(r'\s', '-',track_name)
    track_az = re.sub(r'\s', '',track_name)
    artist_mmatch = re.sub(r'.And', '',artist_array[0])
    artist_mmatch = re.sub(r'\s|\.', '-',artist_mmatch).replace(",","").lower()
    if artist_array[0].startswith("The "):
        the_array = re.split(r'The ',artist_array[0])
        artist_az = re.sub(r'\'|\s|\.|-|\,', '',the_array[1]).lower()
        artist_metro = re.sub(r'\.', '-',the_array[1]).replace(",","").replace(" ","-").lower()
    else:
        artist_az = re.sub(r'\'|\s|\.|-|\,', '',artist_array[0]).lower()
        artist_metro=artist_song
    url_metro_one = 'http://www.metrolyrics.com/'+track_songmetro+'-lyrics-'+artist_metro+'.html'
    url_metro_two = 'http://www.metrolyrics.com/'+track_songmetro+'-lyrics-'+artist_song+'.html'
    url_songlyrics_one = 'http://www.songlyrics.com/'+artist_song+'/'+track_songmetro+'/'
    url_songlyrics_two = 'http://www.songlyrics.com/'+artist_song+'/'+track_songmetro+'-lyrics/'
    url_mode= 'http://www.lyricsmode.com/lyrics/'+artist_first_letter+'/'+artist_mode+'/'+track_mode+'.html'
    #url_music_match = 'https://www.musixmatch.com/lyrics/' + artist_mmatch + '/' + track_songmetro
    #url_azlyrics="http://www.azlyrics.com/lyrics/"+artist_az+"/"+track_az+".html"
    URLS=[url_metro_one,url_metro_two,url_songlyrics_one,url_songlyrics_two,url_mode]#, url_azlyrics, url_music_match]
    lyriclocations = ['lyrics-body-text', 'lyrics-body-text','songLyricsDiv', 'songLyricsDiv','lyrics_text']#
    #,"azlyrics",'mmatch']
    choice =""
    for u in range(0,len(URLS)):
        lyrics_request = urllib.request.Request(URLS[u],headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0'})
        if u!=2:
            try:
                lyrics_web = urllib.request.urlopen(lyrics_request)
                lyrics_code = lyrics_web.read()
                soup = BeautifulSoup(lyrics_code, 'html.parser')
                if u==5:
                    lyrics_array = soup.find_all("div", class_="")
                elif u == 6:
                    lyrics_array = soup.find_all(class_="mxm-lyrics__content")
                else:
                    lyrics_array = soup.find_all(id=lyriclocations[u])
                lyrics_text=""
                for lyrics in lyrics_array:
                    lyrics_text += lyrics.get_text()
                if lyrics_text != "" and lyrics_text != "\r\n " and lyrics_text != "\n":
                    choice=u
                    break
            except urllib.request.HTTPError as e:
                lyrics_text=""
                pass        
    add_frame = {"artist":artist_raw, "track": track_raw, "lyrics":lyrics_text, "source":choice}
    lyrics_data = lyrics_data.append(add_frame,ignore_index=True)
    if row%50==0 and row!=lower_limit and row!=upper_limit:
        sleep(45)
delta = time.time() - start
print ("features retrieved in %.2f seconds" % (delta,))
lyrics_data.to_csv(lyrics_table)

features retrieved in 2.04 seconds


In [55]:
# Cleaning and Wrangling step
# Begin to join all the csv files together, some may require adding in an additional empty row.
lyrics_all=pd.DataFrame()
lyrics_one = pd.read_csv('lyrics-0-100.csv',  encoding='latin1')
lyrics_two = pd.read_csv('lyrics-100-1000.csv',  encoding='latin1')
lyrics_all = pd.concat([lyrics_one,lyrics_two])
lyrics_new = pd.read_csv('lyrics-1000-1096.csv',  encoding='latin1')
lyrics_all = pd.concat([lyrics_all,lyrics_new])

In [80]:
lyrics_new=pd.read_csv('lyrics-27587-27588.csv',encoding='latin1')
lyrics_all = pd.concat([lyrics_all,lyrics_new])

In [86]:
lyrics_merge=lyrics_all[["artist","lyrics","source","track"]].reset_index()


,index,artist,lyrics,source,track
27586,2698,Alan Jackson,"I know you're leaving, I see the signs\r\nYou'...",4.0,www.memory
27587,0,Augie Rios,NaN,NaN,Ã?Â¿DÃ?Â²nde EstÃ?Â Santa Claus? (Where Is Sa...


In [89]:
lyrics_merge.to_csv('all_lyrics.csv')

In [92]:
#Lyrics Scraping Success
len(lyrics_merge[lyrics_merge.lyrics.notnull()])/len(lyrics_merge)*100

82.9781064230825

In [102]:
#annoying=lyrics_merge[lyrics_merge.lyrics=="\r\n "]
lyrics_merge.iloc[3,2]

'\r\r\nSo long ago\r\r\nWas it in a dream?\r\r\nWas it just a dream?\r\r\nI know, yes I know\r\r\nIt seemed so very real\r\r\nSeemed so real to meTook a walk down the street\r\r\nThrough the heat whispered trees\r\r\nI thought I could hear\r\r\nHear\r\r\nHear\r\r\nHearSomebody call out my name (John)\r\r\nAs it started to rain\r\r\nTwo spirits dancing so strange\r\r\nAh! Bowakawa, pousse pousse\r\r\nAh! Bowakawa, pousse pousse\r\r\nAh! Bowakawa, pousse pousseDream, dream away\r\r\nMagic in the air\r\r\nWas magic in the air?\r\r\nI believe, yes I believe\r\r\nMore I cannot say\r\r\nWhat more can I say? '

In [6]:
billboard_lyrics=pd.read_csv("all_lyrics.csv",  encoding='latin1')

In [101]:
#Find the common words
lyrics_list=billboard_lyrics[billboard_lyrics.lyrics.notnull()].reset_index()
lower_limit =0
upper_limit =len(lyrics_list)
common_words = pd.DataFrame()
rgx = re.compile("(\w[\w']*\w|\w)")
for row in range(lower_limit,upper_limit):
    song_lyrics=lyrics_list.get_value(row,"lyrics")
    words = rgx.findall(song_lyrics)
    word_list = [word.lower() for word in words] 
    word_counts = Counter(word_list)
    add_frame = {"artist":lyrics_list.get_value(row,"artist"), "track":lyrics_list.get_value(row,"track"), "lyrics":song_lyrics,"common words":word_counts.most_common()}
    common_words = common_words.append(add_frame,ignore_index=True)

In [112]:
lyric_count=pd.merge(billboard_lyrics,common_words,how='left', on=['artist', 'track',"lyrics"])

In [114]:
word_counts.most_common

Counter({'all': 1,
         'alone': 1,
         'and': 1,
         'around': 1,
         'at': 5,
         'be': 2,
         'call': 1,
         'can': 2,
         'click': 3,
         'day': 1,
         'even': 2,
         'eyes': 1,
         'feel': 4,
         'for': 2,
         'gone': 1,
         'gonna': 1,
         'hand': 1,
         'have': 2,
         'heart': 1,
         'hold': 1,
         'i': 2,
         "i'll": 2,
         'if': 4,
         'into': 1,
         'it': 1,
         'just': 4,
         'keyboard': 1,
         'know': 1,
         'land': 1,
         'leaving': 1,
         'like': 2,
         'little': 1,
         'log': 1,
         'look': 1,
         'love': 2,
         'made': 1,
         'me': 9,
         'memory': 5,
         'mind': 1,
         'mine': 1,
         'mouse': 1,
         'my': 2,
         'need': 2,
         'never': 2,
         'no': 1,
         'of': 1,
         'on': 5,
         'or': 2,
         'out': 1,
         'patiently': 2,
      

In [116]:
if 'you' in word_counts:
    print(word_counts["you"])

3


In [143]:
#sad_songs = merged_stolen[merged_stolen.mode==1 and merged_stolen.key==1]

In [75]:
song_lyrics=billboard_lyrics.get_value(11,"lyrics")
print(song_lyrics)
#words = rgx.findall(song_lyrics)
#word_list = [word.lower() for word in words] 
#word_counts = Counter(word_list)
#word_counts
#    add_frame = {"common words":word_counts.most_common()}
#    common_words = common_words.append(add_frame,ignore_index=True)

nan


In [309]:
test_1[test_1.lyrics=="\r\n "]

,Unnamed: 0,artist,lyrics,source,track
18,18,Suzi Quatro,\r\n,0.0,All Shook Up
260,260,Roger Williams,\r\n,0.0,Amor
280,280,Esther Phillips,\r\n,0.0,And I Love Him
294,294,Mark Lindsay,\r\n,0.0,And The Grass Won't Pay No Mind
394,394,Jerry Reed,\r\n,0.0,Another Puff
513,513,Mark Lindsay,\r\n,0.0,Are You Old Enough
575,575,"Ecstasy, Passion & Pain",\r\n,0.0,Ask Me
591,591,T-Connection,\r\n,0.0,At Midnight
738,738,Idina Menzel Duet With Michael Buble,\r\n,0.0,Baby It's Cold Outside
745,745,King Floyd,\r\n,0.0,Baby Let Me Kiss You


In [18]:
merged_stolen = pd.merge(new_dat,audio_check,left_on="spotifyID",right_on="id")

In [41]:
true_dat=merged_stolen[merged_stolen.artist=="Adele"][["track_href"]]

In [166]:
lyrics_data = pd.DataFrame()
lower_limit =0
upper_limit =333
lyrics_table = 'lyricstwo-' +str(lower_limit)+"-"+ str(upper_limit)+'.csv'
start = time.time()
for row in range(lower_limit,upper_limit):
    artist_raw = empty_table.iloc[row,1]
    artist_noand=artist_raw.replace(" &","").replace(". ","-")
    artist_array = re.split(r'.Featuring+', artist_noand)
    track_raw=empty_table.iloc[row,4]
    track_name=re.sub(r'\&\s', '', track_raw).replace("$", "s").replace("#","") # Replace certain characters
    track_name=re.sub(r'S\*\*t', 'Shit',track_name) # Replace profanity
    track_name=re.sub(r'\(|\)|\'|\|\.|\"', '',track_name).lower() # Remove special characters
    # We try four different lyrics sites
    artist_first_letter=artist_array[0][0]
    artist_mode = re.sub(r'\s|\.|-', '_',artist_array[0]).replace(",","").lower()
    track_mode = re.sub(r'\s', '_',track_name)
    artist_song = re.sub(r'\s|\.', '-',artist_array[0]).replace(",","").lower()
    track_songmetro = re.sub(r'\s', '-',track_name)
    track_az = re.sub(r'\s', '',track_name)
    artist_mmatch = re.sub(r'.And', '',artist_array[0])
    artist_mmatch = re.sub(r'\s|\.', '-',artist_mmatch).replace(",","").lower()
    if artist_array[0].startswith("The"):
        the_array = re.split(r'The ',artist_array[0])
        artist_az = re.sub(r'\'|\s|\.|-|\,', '',the_array[1]).lower()
        artist_metro = re.sub(r'\.', '-',the_array[1]).replace(",","").replace(" ","-").lower()
    else:
        artist_az = re.sub(r'\'|\s|\.|-|\,', '',artist_array[0])
        artist_metro=artist_song
    url_metro_one = 'http://www.metrolyrics.com/'+track_songmetro+'-lyrics-'+artist_metro+'.html'
    url_metro_two = 'http://www.metrolyrics.com/'+track_songmetro+'-lyrics-'+artist_song+'.html'
    url_songlyrics_one = 'http://www.songlyrics.com/'+artist_song+'/'+track_songmetro+'/'
    url_songlyrics_two = 'http://www.songlyrics.com/'+artist_song+'/'+track_songmetro+'-lyrics/'
    url_mode= 'http://www.lyricsmode.com/lyrics/'+artist_first_letter+'/'+artist_mode+'/'+track_mode+'.html'
    url_music_match = 'https://www.musixmatch.com/lyrics/' + artist_mmatch + '/' + track_songmetro
    url_azlyrics="http://www.azlyrics.com/lyrics/"+artist_az+"/"+track_az+".html"
    URLS=[url_metro_one,url_metro_two,url_songlyrics_one,url_songlyrics_two,url_mode, url_azlyrics, url_music_match]
    lyriclocations = ['lyrics-body-text', 'lyrics-body-text','songLyricsDiv', 'songLyricsDiv','lyrics_text',"azlyrics",'mmatch']
    choice =""
    for u in range(0,len(URLS)):
        lyrics_request = urllib.request.Request(URLS[u],headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:52.0) Gecko/20100101 Firefox/52.0'})
        if u==3 or u==6:
            try:
                lyrics_web = urllib.request.urlopen(lyrics_request)
                lyrics_code = lyrics_web.read()
                soup = BeautifulSoup(lyrics_code, 'html.parser')
                if u==5:
                    lyrics_array = soup.find_all("div", class_="")
                elif u == 6:
                    lyrics_array = soup.find_all(class_="mxm-lyrics__content")
                else:
                    lyrics_array = soup.find_all(id=lyriclocations[u])
                lyrics_text=""
                for lyrics in lyrics_array:
                    lyrics_text += lyrics.get_text()
                if lyrics_text != "" and lyrics_text != "\r\n " and lyrics_text != "\n":
                    choice=u
                    break
            except urllib.request.HTTPError as e:
                lyrics_text=""
                pass
    add_frame = {"artist":artist_raw, "track": track_raw, "lyrics":lyrics_text, "source":choice}
    lyrics_data = lyrics_data.append(add_frame,ignore_index=True)
    if row%50==0 and row!=lower_limit and row!=upper_limit:
        sleep(45)
delta = time.time() - start
print ("features retrieved in %.2f seconds" % (delta,))
#lyrics_data.to_csv(lyrics_table)

features retrieved in 759.40 seconds
